In [130]:
import numpy as np
import pandas as pd
import nltk
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import re
import random
import collections
# use multiprocessing for multicore ops
import copy
#nltk.download()
from nltk.corpus import stopwords # Import the stop word list
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier


def getSimilarity(df, cv):
    r,c = df.shape
    #print(str(r)+ str(c))
    final_vec = []
    for j in range(r):
        a = cv.transform([df.iloc[j].question1])
        b = cv.transform([df.iloc[j].question2])
        sim = round(cosine_similarity(a,b).ravel()[0], 3)
        c = df.iloc[j].is_duplicate
        final_vec.append([sim, c])
    return final_vec

def question_to_words(raw_quest, lower=True, remove_not=True):
    lemmetizer = nltk.WordNetLemmatizer()

    letters_only = re.sub("[^a-zA-Z]", " ", raw_quest)
    if lower == True:
        letters_only = letters_only.lower()
    words = nltk.word_tokenize(letters_only)
    # words = nltk.word_tokenize(s)
    stops = set(stopwords.words("english"))
    not_stop = ['no', 'nor', 'not', "don't", "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't",
                'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn',
                "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren',
                "weren't", 'won', "won't", 'wouldn', "wouldn't"]
    if remove_not == True:
        # stop = stops.difference(not_stop)
        stops = ['the', 'a', 'an', 'and', 'but', 'if', 'or', 'because', 'as', 'what', 'which', 'this', 'that', 'these',
                 'those', 'then',
                 'just', 'so', 'than', 'such', 'both', 'through', 'about', 'for', 'is', 'of', 'while', 'during', 'to',
                 'What', 'Which',
                 'Is', 'If', 'While', 'This']
    meaningful_words = [w for w in words if not w in stops]

    meaningful_words = [lemmetizer.lemmatize(w) for w in meaningful_words]
    return (" ".join(meaningful_words))


def get_cosine_df(X_train, y_train):
    X_train['question1'] = X_train['question1'].apply(question_to_words)
    X_train['question2'] = X_train['question2'].apply(question_to_words)
    #print(X_train.shape)
    #print(X_train.head)

    #print("Creating the bag of words...\n")

    vectorizer = CountVectorizer(analyzer="word", \
                                 tokenizer=None, \
                                 preprocessor=None, \
                                 stop_words=None, \
                                 ngram_range=(1,2),\
                                 max_features=5000)
    corpus = X_train["question1"] + " " + X_train["question2"]
    cv = vectorizer.fit(corpus)

    # cv
    #print(X_train)
    #print(y_train)
    new_data = pd.concat([X_train, y_train], axis=1, join_axes=[X_train.index])
    #print(new_data.head)

    duplicates = new_data[new_data.is_duplicate == 1]
    nonduplicates = new_data[new_data.is_duplicate == 0]
    #print(duplicates.shape)
    #print(nonduplicates.shape)

    sim = getSimilarity(new_data, cv)
    return sim


data2 = pd.read_csv("../data/questions.csv", delimiter=',', encoding="utf-8-sig")
data1 = copy.deepcopy(data2[:50000])
data1.dropna(inplace=True)
#print(data1.shape)
#print(data1.iloc[0:1, 3:])

X = data1[["question1", "question2"]]
y = data1[['is_duplicate']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#print(stopwords.words("english"))

#print(X_train.shape)

sim = get_cosine_df(X_train, y_train)
df1 = pd.DataFrame(sim)
df1.columns = ["cos_sim", "y"]
#df1




/home/ss/.local/lib/python3.5/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ss/.local/lib/python3.5/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [131]:
X = df1['cos_sim']
y = df1['y']
X = X.values.reshape(-1, 1)
y = y.values.reshape(-1,1)
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=2)
neigh.fit(X,y)


/home/ss/.local/lib/python3.5/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=2, p=2,
           weights='uniform')

In [132]:
print(X_test.shape)
sim = get_cosine_df(X_test, y_test)
df1 = pd.DataFrame(sim)
df1.columns = ["cos_sim", "y"]
#df1
#print(df1.head)
Xt = df1['cos_sim']
yt = df1['y']
Xt = Xt.values.reshape(-1, 1)
p = neigh.predict(Xt)

(16500, 2)


In [133]:
p

array([0, 0, 0, ..., 0, 0, 0])

In [134]:
n = p.tolist()
print(len(n))
#n

16500


In [135]:
fact = yt.tolist()
print(len(fact))
#fact

16500


In [136]:

assert(len(fact)==len(p))
count = 0
for i in range(len(fact)):
    if fact[i]==p[i]:
        count+=1
print(count)
print("Accuracy = " + str(count/len(fact)) )

10399
Accuracy = 0.6302424242424243
